In [1]:
import sys

class Person:
    pass

p1 = Person()

print(sys.getrefcount(p1))  # 1

p2 = p1   # 2

print(sys.getrefcount(p1))

del p2   # 1
print(sys.getrefcount(p1))

del p1   # 0 但是测不出来
print(sys.getrefcount(p1))


2
3
2


NameError: name 'p1' is not defined

### 引用计数 +1 的场景:

1. 对象被创建: p1 = Person()
2. 对象被引用: p2 = p1
3. 对象被作为参数，传入到一个函数中
4. 对象作为一个元素，存储在容器中: l = [p1]

#### getrefcount 会大1

In [2]:
class Person:
    pass

pxxx = Person()  # 1
print(sys.getrefcount(pxxx))

def log(obj):
    print(sys.getrefcount(obj))

log(pxxx)

2
4


In [3]:
l = [pxxx]
print(sys.getrefcount(pxxx))

3


### 引用计数 -1 的场景:

1. 对象的别名被显示销毁： del p1
2. 对象的别名被赋予新的对象： p1 = 123
3. 一个对象离开它的作用域：
    1. 一个函数执行完毕时
    2. 内部的局部变量关联的对象，它的引用计数会 -1
4. 对象所在的容器被销毁，或从容器中删除对象

### 循环引用

❌循环引用的无法用 引用计数器回收  
但是Python还有垃圾回收机制~

In [4]:
import objgraph
# objgraph.count() 可以查看垃圾回收器，跟踪的对象个数

In [5]:
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

print(objgraph.count("Person"))

del p
del d

print(objgraph.count("Person"))

2
1


In [6]:
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

print(objgraph.count("Person"))

p.pet = d       # 注意这里
d.master = p    # 循环引用

del p
del d

print(objgraph.count("Person"))  # 没有释放❌

2
2


### 垃圾回收机制

找到循环引用，干掉相关对象

1. 收集所有的“容器对象”，通过一个双向链表进行引用
  1. 容器对象： 列表，元组，字典，自定义类对象
2. 针对每一个“容器对象”，通过一个变量gc_refs来记录当前对应的引用计数
3. 对于每个“容器对象”，找到它引用的“容器对象”，并将这个“容器对象”的引用计数-1
4. 经过步骤3，如果一个“容器对象”的引用计数为0，就代表这玩意可以被回收了，  
肯定是“循环引用”导致他活到现在的

其他 见思维导图

In [7]:
import gc
print(gc.get_threshold())
# 调大 检测频率减少，提升程序性能，因为垃圾回收比较耗时
# eg(700, 10, 5) 当垃圾回收器中，新增对象个数- 消亡的对象个数 = 700时会触发
# 分代回收机制：0代检测超过10次，才会触发1代检测；
# 1代触发超过5次，才会触发0代和1代和2代的检测

(700, 10, 10)


In [8]:
gc.set_threshold(200, 5, 5)

In [9]:
gc.disable()
print(gc.isenabled())

False


In [10]:
gc.collect()

40

In [20]:
# 手动触发
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

print(objgraph.count("Person"))

p.pet = d       # 注意这里
d.master = p    # 循环引用

del p
del d

gc.collect() # 不关心垃圾回收机制是否开启

print(objgraph.count("Person"))  # 理论上这里应该是0 有点问题不知道为啥


2
1


#### 避免内存泄漏：“预防”:1.弱引用

In [21]:
# 弱引用
import weakref
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

print(objgraph.count("Person"))

p.pet = d       # 注意这里
d.master = weakref.ref(p)    # 循环引用

del p
del d

gc.collect() # 不关心垃圾回收机制是否开启

print(objgraph.count("Person"))  # 理论上这里应该是0 有点问题不知道为啥

2
1


#### “治疗”：删除引用时，手动清空对其他容器对象的引用

In [22]:
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

print(objgraph.count("Person"))

p.pet = d       # 注意这里
d.master = p    # 循环引用

p.pet = None

del p
del d

gc.collect() # 不关心垃圾回收机制是否开启

print(objgraph.count("Person"))  # 理论上这里应该是0 有点问题不知道为啥
print(objgraph.count("Dog"))

2
1
0


```
p.pet = {"dog": weakref.ref(d1), "cat": weakref.ref(c1)}     # 注意这里，写法1
# weakref.WeakValueDictionary({"dog": d1, "cat": c1})        # 写法2
```